In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown
from plotly import graph_objects as go

import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_pop_death_data, get_death_rates
from tb_incubator.model import build_model
from tb_incubator.input import load_param_info

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

## Population data

We used estimates for the total population, deaths, and birth rate from the United Nations World Population Prospects [@unwpp2024].

In [ ]:
# Load age-stratified, population and death data
pop_death, description = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]
Markdown(description)

## Model construction
This is arbitrary, but including some epidemiological transitions in the background.

In [3]:
compartments = const.compartments
infectious_compartments = const.infectious_compartments
age_strata = const.age_strata
model_times = const.model_times

In [ ]:
param_info = load_param_info()
params = param_info["value"]

model, desc = build_model(
    compartments,
    infectious_compartments,
    age_strata,
    params,
    model_times
)
Markdown(desc)

In [5]:
test_params = {
	'contact_rate': 9.543921666073745,
	'self_recovery_rate': 0.29688347664390236,
	'screening_scaleup_shape': 0.19889392155636598,
	'screening_inflection_time': 2009.3016735363497,
	'time_to_screening_end_asymp': 0.3361714758423252,
	'rr_infection_latent': 0.116003030825429,
	'rr_infection_recovered': 0.8720080623263372,
	'seed_time': 1881.49881702568,
	'seed_duration': 19.549815951426623,
	'seed_rate': 70.64648160388384,
	'base_sensitivity': 0.2867398245514573,
	'genexpert_sensitivity': 0.826222381252117,
	'progression_multiplier': 1.436023746975478,
}


In [6]:
# age-stratified population
age_pops_output = [model.request_output_for_compartments(f"total_populationXage_{s}", compartments, strata={"age": str(s)}) for s in age_strata]


In [7]:
model.run(params | test_params)

## Demographic results

In [ ]:
# Run and inspect results
model.run(params)
fig = model.get_derived_outputs_df()[[f"total_populationXage_{age}" for age in age_strata]].plot.area()
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))